<a href="https://colab.research.google.com/github/VISHAL-ULTIMUS/BIOINFORMATICS/blob/main/IBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import itertools
from tkinter import *
from tkinter import ttk
import pylab
import numpy as np
import matplotlib.pyplot as plt
import keras
import numpy as np
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import matplotlib.pyplot as plt
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image


!pip install biopython
!pip install plotting




     |████████████████████████████████| 2.3MB 9.0MB/s 


In [ ]:
from Bio import SeqIO
covid = SeqIO.read("/content//covid.fasta","fasta")
mers = SeqIO.read("/content//mers.fasta","fasta")
sars = SeqIO.read("/content//sars.fasta","fasta")

In [ ]:
covid_seq = covid.seq
mers_seq = mers.seq
sars_seq = sars.seq

NameError: ignored

In [ ]:
print("covid_seq ::",len(covid_seq))
print("mers_seq ::",len(mers_seq))
print("sars_seq ::",len(sars_seq))

NameError: ignored

In [ ]:
from Bio.SeqUtils import GC


In [ ]:
print("GC content of covid_seq ::",GC(covid_seq))
print("GC content of mers_seq ::",GC(mers_seq))
print("GC content of sars_seq ::",GC(sars_seq))

GC content of covid_seq :: 37.97277865097148
GC content of mers_seq :: 41.23642883229855
GC content of sars_seq :: 40.7616550704178


In [ ]:
mersfile = "/content/mers.fasta"#MERS
covidfile = "/content/covid.fasta"#COVID
sarsfile = "/content/sars.fasta"#SARS

outfile = "/content/sample_data/output"

sequence = ""
with open(mersfile, 'r') as f:
    for line in f:
        if line.startswith('>'):
            seq_id = line.rstrip()[0:]
        else:
            sequence += line.rstrip()
GC_content = float((sequence.count('G') + sequence.count('C'))) / len(sequence) * 100
with open(outfile, 'a') as file_out:
    file_out.write("The GC content of '%s' is\t %.2f%%" % (seq_id, GC_content))
print("GC content in MERS is")
print(GC_content)

sequence2 = ""
with open(covidfile, 'r') as f:
    for line in f:
        if line.startswith('>'):
            seq_id = line.rstrip()[0:]
        else:
            sequence2 += line.rstrip()
GC_content = float((sequence2.count('G') + sequence2.count('C'))) / len(sequence2) * 100
with open(outfile, 'a') as file_out:
    file_out.write("The GC content of '%s' is\t %.2f%%" % (seq_id, GC_content))
print("GC content in COVID is")
print(GC_content)

sequence3 = ""
with open(sarsfile, 'r') as f:
    for line in f:
        if line.startswith('>'):
            seq_id = line.rstrip()[0:]
        else:
            sequence3 += line.rstrip()
GC_content = float((sequence3.count('G') + sequence3.count('C'))) / len(sequence3) * 100
with open(outfile, 'a') as file_out:
    file_out.write("The GC content of '%s' is\t %.2f%%" % (seq_id, GC_content))
print("GC content in SARS is")
print(GC_content)
print(sequence3)         
print(sequence2)         


GC content in MERS is
41.23642883229855
GC content in COVID is
37.97277865097147
GC content in SARS is
40.7616550704178
...


## Alignment Types

Global alignment: finds the best concordance/agreement betwenn all characters in two sequences
Local Alignment: finds just the subsequences that align the best

In [ ]:

def matrix(a, b, match_score=3, gap_cost=2):
    H = np.zeros((len(a) + 1, len(b) + 1), np.int)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else - match_score)
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H
def traceback(H, b, b_='', old_i=0):
    # flip H to get index of **last** occurrence of H.max() with np.argmax()
    H_flip = np.flip(np.flip(H, 0), 1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1))  # (i, j) are **last** indexes of H.max()
    if H[i, j] == 0:
        return b_, j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_
    return traceback(H[0:i, 0:j], b, b_, i)
def smith_waterman(a, b, match_score=3, gap_cost=2):
    a, b = a.upper(), b.upper()
    H = matrix(a, b, match_score, gap_cost)
    b_, pos = traceback(H, b)
    return pos, pos + len(b_)

In [ ]:
  print(matrix(sequence[0:50], sequence2[0:50]))

  a, b = sequence[0:50], sequence2[0:50]
  H  = matrix(a, b)
  
  print(traceback(H, b))
  a, b = sequence[0:50], sequence2[0:50]

  start, end = smith_waterman(a, b)
  print(a[start:end])   

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  3  1 ...  2  0  0]
 ...
 [ 0  3  1 ... 30 32 37]
 [ 0  1  0 ... 28 30 35]
 [ 0  3  1 ... 27 28 33]]
('ATT-AAG-G-TA-CTT-CCC-GT-CTC', 0)
GATTTAAGTGAATAGCTTGGCTATCTC


##Hamming distance: shows how many places 2 strings differ

Hamming distance between two strings of equal length is the number of positions at which the corresponding symbols are different.
In other words, it measures the minimum number of substitutions required to change one string into the other, or the minimum number of errors that could have transformed one string into the othe
It is used for error detection or error correction
It is used to quantify the similarity of DNA sequences,
For checking the edit distance
edit distance is a way of quantifying how dissimilar two strings (e.g., words) are to one another by counting the minimum number of operations required to transform one string into the other.
eg Levenshtein distance


In [ ]:
def hammingDist(str1, str2):
    i = 0
    count = 0
 
    while(i < len(str1)):
        if(str1[i] != str2[i]):
            count += 1
        i += 1
    return count
print(hammingDist(sequence[0:50], sequence2[0:50]))
print(hammingDist(sequence[0:50], sequence3[0:50]))

35
33


### Transcription

In [ ]:
dna = sequence
new = ""

for i in dna:
    if i not in 'ATGC':
        new = "Invalid Input"
        break
    if i == 'A':
        new += 'U'
    elif i == 'C':
        new += 'G'
    elif i == 'T':
        new += 'A'
    else:
        new += 'C'

print(new)

...



### Translation


In [ ]:
rna = new

print ("RNA String: ", rna)

# RNA codon table
rna_codon = {"UUU" : "F", "CUU" : "L", "AUU" : "I", "GUU" : "V",
           "UUC" : "F", "CUC" : "L", "AUC" : "I", "GUC" : "V",
           "UUA" : "L", "CUA" : "L", "AUA" : "I", "GUA" : "V",
           "UUG" : "L", "CUG" : "L", "AUG" : "M", "GUG" : "V",
           "UCU" : "S", "CCU" : "P", "ACU" : "T", "GCU" : "A",
           "UCC" : "S", "CCC" : "P", "ACC" : "T", "GCC" : "A",
           "UCA" : "S", "CCA" : "P", "ACA" : "T", "GCA" : "A",
           "UCG" : "S", "CCG" : "P", "ACG" : "T", "GCG" : "A",
           "UAU" : "Y", "CAU" : "H", "AAU" : "N", "GAU" : "D",
           "UAC" : "Y", "CAC" : "H", "AAC" : "N", "GAC" : "D",
           "UAA" : "STOP", "CAA" : "Q", "AAA" : "K", "GAA" : "E",
           "UAG" : "STOP", "CAG" : "Q", "AAG" : "K", "GAG" : "E",
           "UGU" : "C", "CGU" : "R", "AGU" : "S", "GGU" : "G",
           "UGC" : "C", "CGC" : "R", "AGC" : "S", "GGC" : "G",
           "UGA" : "STOP", "CGA" : "R", "AGA" : "R", "GGA" : "G",
           "UGG" : "W", "CGG" : "R", "AGG" : "R", "GGG" : "G" 
           }

protein_string = ""

# Generate protein string
for i in range(0, len(rna)-(3+len(rna)%3), 3):
    if rna_codon[rna[i:i+3]] == "STOP" :
        break
    protein_string += rna_codon[rna[i:i+3]]

# Print the protein string
print ("Protein String: ", protein_string)



RNA String:  ...
Protein String:  LNSLIEPIE


### DOT MATRIX

In [ ]:

seq1 = sequence[0:50]
seq2 = sequence2[0:50]

dot_matrix = np.zeros((len(seq1),len(seq2)))
for i in range(len(seq1)):
  for j in range  (len(seq2)):
    if seq1[i] == seq2[j]:
      dot_matrix[i][j]= 1 
    else:
        dot_matrix[i][j]= 0
dot_matrix

dotplot=plt.imshow(dot_matrix)
xt=plt.xticks(np.arange(len(list(seq2))),list(seq2))
yt=plt.yticks(np.arange(len(list(seq1))),list(seq1))
plt.show()

var_matrix = dot_matrix
print('matrix for the sequence:\n',var_matrix)
max_col = len(var_matrix[0])
max_row = len(var_matrix)
columns = [[]for i in range(max_col)]
rows = [[]for i in range(max_row)]
diagonal_ele = [[]for i in range(max_row + max_col - 1)]
rev_diagonal =  [[]for i in range(len(diagonal_ele))]
min_rev_diagonal = -max_row + 1

for x in range(max_col):
  for y in range(max_row):
    diagonal_ele[x+y].append(var_matrix[y][x]) 
    rev_diagonal[x-y-min_rev_diagonal].append(var_matrix[y][x])   
print('forward diagonals : \n',diagonal_ele)
print('reverse diagonals :\n', rev_diagonal)

max_sum1 = sum(diagonal_ele[0])
position = 1
for i,j in enumerate(diagonal_ele):
  if sum(diagonal_ele[i]) > max_sum1:
    max_sum1 = sum(diagonal_ele[i])
    position = i


max_sum2 = sum(diagonal_ele[0])
position2 = 1
for k,j in enumerate(rev_diagonal):
  if sum(rev_diagonal[k]) > max_sum2:
    max_sum2 = sum(rev_diagonal[k])
    position2 = k


if (max_sum1>=max_sum2):
  print('matches for the sequences '. max_sum1)
  print('position of the longest diagonal from forward diagonal',position1)
  print(diagonal_ele[position1])
else :
  print('matches for the sequences ', max_sum2)
  print('position of the longest diagonal  from reverse diagonal',position2)
  print(rev_diagonal[position2])



NameError: ignored

In [ ]:
cov_n_sars = pairwise2.align.globalxx(sequence[0:100],sequence2[0:100],one_alignment_only=True,score_only=True)
print(cov_n_sars)
print(cov_n_sars/len(sequence2[0:100]) * 100)
cov_n_mers = pairwise2.align.globalxx(sequence[0:100],sequence3[0:100],one_alignment_only=True,score_only=True)
print(cov_n_mers)
print(cov_n_mers/len(sequence3[0:100]) * 100)


71.0
71.0
69.0
69.0


In [ ]:


TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Test"






## 2. Buiding the CNN Architecture



# CNN Based Model in Keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])


model.summary()


## 3. Data Augmentation, Normalizing Data & Training the Model


# Train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')




train_generator.class_indices



validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')


hist = model.fit_generator(
    train_generator,
    steps_per_epoch=7,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=2
)


model.save("model_adv.h5")



model.evaluate_generator(train_generator)


model.evaluate_generator(validation_generator)


## 4. Model Evaluation


model = load_model('model_adv.h5')



import os



train_generator.class_indices



y_actual = []
y_test = []



from PIL import Image
for i in os.listdir("./CovidDataset/Val/Normal/"):
  img = image.load_img("./CovidDataset/Val/Normal/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_classes(img)
  y_test.append(p[0,0])
  y_actual.append(1)



for i in os.listdir("./CovidDataset/Val/Covid/"):
  img = image.load_img("./CovidDataset/Val/Covid/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_classes(img)
  y_test.append(p[0,0])
  y_actual.append(0)


### 4.1 Confusion Matrix


y_actual = np.array(y_actual)
y_test = np.array(y_test)



from sklearn.metrics import confusion_matrix


cm = confusion_matrix(y_actual, y_test)



import seaborn as sns
sns.heatmap(cm, cmap=None, annot=True)


### 4.2 Plotting Training & Validation Loss


import matplotlib.pyplot as plt
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


### 4.3 Plotting Training & Validation Accuracy



acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
plt.plot(epochs, acc, color='red', label='Training acc')
plt.plot(epochs, val_acc, color='green', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


## 5. Some Comparative Results of Radiologist vs CNN Model on Validation set


class color:
   GREEN = '\033[92m'
   RED = '\033[91m'
   
from PIL import Image
j = 0
for i in os.listdir("./CovidDataset/Val/Normal/"):
  j = j+1
  pil_img = Image.open('./CovidDataset/Val/Normal/'+i)
  im = pil_img
  size = (200,200) #image size
  
  out = im.resize(size)
  display(out)
  img = image.load_img("./CovidDataset/Val/Normal/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_classes(img)
  print(color.GREEN + 'Radiologist: COVID-19 -ve' + color.GREEN)

  if p==0:
    print(color.RED + 'CNN Model: COVID-19 +ve' + color.RED)
  else:
    print("CNN Model: COVID-19 -ve")
  if (j==5):
    break

from PIL import Image
j = 0
for i in os.listdir("./CovidDataset/Val/Covid/"):
  j = j+1
  pil_img = Image.open('./CovidDataset/Val/Covid/'+i)
  im = pil_img
  size = (200,200) #image size
  
  out = im.resize(size)
  display(out)
  img = image.load_img("./CovidDataset/Val/Covid/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_classes(img)
  print(color.RED + 'Radiologist: COVID-19 +ve' + color.RED)
  if p==0:
    print("CNN Model: COVID-19 +ve")
  else:
    print(color.GREEN + 'Radiologist: COVID-19 -ve' + color.GREEN)
  if (j==5):
    break




